In [10]:
#Read in Redlist

import pandas as pd
import py_entitymatching as em
import math
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import style
style.use('ggplot')

# Read in csv as dataframe
rl = em.read_csv_metadata("tenFeaturesRedlist.csv", encoding="ISO-8859-1", key='id')
# glance at first few rows
rl.head()


,id,name,genus,family,ecology,countries,threat_paragraph,conservation_paragraph,pop_trend,status,country_count
0,0,Ochlockonee Moccasinshell,Medionidus,Unionidae,Freshwater,"United States (Florida, Georgia);",NaN,NaN,NaN,\n Critically Endangered\n\n\n A1ce\n\n,1
1,1,Nelson's Spiny Pocket Mouse,Heteromys,Heteromyidae,Terrestrial,Guatemala; Mexico (Chiapas);,\n The main threat to this species is the significant forest loss in its habitat. Habitat fra...,"['\n This species occurs in a newly-named national park, Tacana, in Mexico.\n\n \n ']",Decreasing,"\n Endangered\n\n\n B1ab(i,ii,iii,v)\n\n",2
2,2,Comoro Friar,Amauris,Nymphalidae,NaN,Comoros;,NaN,NaN,NaN,"\n Endangered\n\n\n B1+2c, C2b\n\n",1
3,3,"Atlantic Halibut, Halibut",Hippoglossus,Pleuronectidae,NaN,<div>Atlantic  northeast; Atlantic  northwest</div>; Canada (Newfoundland I); Denmark; Far...,NaN,NaN,NaN,\n Endangered\n\n\n A1d\n\n,15
4,10,Oktibbeha Riverlet Crayfish,Hobbseus,Cambaridae,Freshwater,United States (Mississippi);,\n This species is threatened by habitat degradation through residential and urban developmen...,"['\n', <span style=""font-style: italic;""></span>, 'This species has been given the Global Herita...",Unknown,\n Endangered\n\n\n B1ab(iii)\n\n,1


In [ ]:
# CME
em.get_key(rl) # get key

# Note: The id column of the  redlist csv doesn't actually look like the object created here (rl).  In the csv,
# id numbers jump around since they represent original row number of the original scraped data csv.  Apparently
# this library can detect what we want and enumerate things in order if you give it a key feature like this.


In [20]:
#Read in ARKIVE
# Read in csv as dataframe
ar = em.read_csv_metadata("arkive table analysis/output.csv", encoding="ISO-8859-1", key='id')

# glance at first few rows
ar.head()

,id,scientific_name,name,kingdom,phylum,class,order,family,genus,size,...,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21
0,0,Alcatraz de Abbott,Piquero de Abbott,Abbott's booby,Animalia,Chordata,Aves,Pelecaniformes,Sulidae,Papasula (1),...,The majority of Abbott's booby nest sites occur within the Christmas Island National Park which ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,AbbotÈs duiker,AbbottÈs duiker,Animalia,Chordata,Mammalia,Cetartiodactyla,Bovidae,Cephalophus (1),50  60 kg (2),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Cisticole des Aberdares,Aberdare cisticola,Animalia,Chordata,Aves,Passeriformes,Cisticolidae,Cisticola (1),Cisticola (1),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Adelaide pigmy blue-tongue skink,pygmy bluetongue,Adelaide pygmy bluetongue skink,Animalia,Chordata,Reptilia,Squamata,Scincidae,Tiliqua (1),...,The South Australia Department of Environment and Conservation has produced a National Recovery ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Adriatic trout,softmouth trout,Adriatic salmon,Animalia,Chordata,Actinopterygii,Salmoniformes,Salmonidae,Salmo (1),...,There are currently no known conservation measures in place for the Adriatic salmon (1) (5). It ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,Bejeque gomero,Aeonium,Plantae,Tracheophyta,Magnoliopsida,Rosales,Crassulaceae,Aeonium (1),Aeonium (1),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,black-footed penguin,jackass penguin,African penguin,Animalia,Chordata,Aves,Sphenisciformes,Spheniscidae,Spheniscus (1),...,The African penguin is protected by its listing on Appendix II of the Convention on Internationa...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,Afrormosia,Assamela,Kokrodua,African teak,Plantae,Tracheophyta,Magnoliopsida,Fabales,Leguminosae,...,The trade in the timber from this tree has soared since 1948 and the degree of exploitation has ...,Urgent regulation of logging is necessary including the employment of Forestry Stewardship Counc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,Cape hunting dog,painted hunting dog,African wild dog,Animalia,Chordata,Mammalia,Carnivora,Canidae,Lycaon (1),...,The current population of African wild dogs is estimated to be less than 5500 individuals (2). T...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,Kaori de montagne,Kaori nain,Agathis,Plantae,Tracheophyta,Coniferopsida,Coniferales,Araucariaceae,Agathis (1),...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:

# Remove parentheticals from arkive genus
# author: @andrewedstrom
# Next, actually remove scientific name and replace with common name(s)
#ar = arBeforeRemovingNames # in case script now run completely in order
#ar = ar[ar.animalName.str.contains("\(") == True] # update df to contain only tuples that contain common/nick names
error = 0 # make sure there's no wonky cases

# remove parenthesis contents
for index, row in ar.iterrows():
    genus = str(row['genus'])

    pstart = genus.find("(")
    if pstart !=-1:        
        # extract full common/nick name
        noparens = genus[0:pstart]
        ar.loc[index, 'genus'] = noparens
    else:
        error = error + 1


print(error)        
ar

275


,id,scientific_name,name,kingdom,phylum,class,order,family,genus,size,...,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21
0,0,Alcatraz de Abbott,Piquero de Abbott,Abbott's booby,Animalia,Chordata,Aves,Pelecaniformes,Sulidae,Papasula (1),...,The majority of Abbott's booby nest sites occur within the Christmas Island National Park which ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,AbbotÈs duiker,AbbottÈs duiker,Animalia,Chordata,Mammalia,Cetartiodactyla,Bovidae,Cephalophus,50  60 kg (2),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Cisticole des Aberdares,Aberdare cisticola,Animalia,Chordata,Aves,Passeriformes,Cisticolidae,Cisticola,Cisticola (1),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Adelaide pigmy blue-tongue skink,pygmy bluetongue,Adelaide pygmy bluetongue skink,Animalia,Chordata,Reptilia,Squamata,Scincidae,Tiliqua (1),...,The South Australia Department of Environment and Conservation has produced a National Recovery ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Adriatic trout,softmouth trout,Adriatic salmon,Animalia,Chordata,Actinopterygii,Salmoniformes,Salmonidae,Salmo (1),...,There are currently no known conservation measures in place for the Adriatic salmon (1) (5). It ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,Bejeque gomero,Aeonium,Plantae,Tracheophyta,Magnoliopsida,Rosales,Crassulaceae,Aeonium,Aeonium (1),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,black-footed penguin,jackass penguin,African penguin,Animalia,Chordata,Aves,Sphenisciformes,Spheniscidae,Spheniscus (1),...,The African penguin is protected by its listing on Appendix II of the Convention on Internationa...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,Afrormosia,Assamela,Kokrodua,African teak,Plantae,Tracheophyta,Magnoliopsida,Fabales,Leguminosae,...,The trade in the timber from this tree has soared since 1948 and the degree of exploitation has ...,Urgent regulation of logging is necessary including the employment of Forestry Stewardship Counc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,Cape hunting dog,painted hunting dog,African wild dog,Animalia,Chordata,Mammalia,Carnivora,Canidae,Lycaon (1),...,The current population of African wild dogs is estimated to be less than 5500 individuals (2). T...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,Kaori de montagne,Kaori nain,Agathis,Plantae,Tracheophyta,Coniferopsida,Coniferales,Araucariaceae,Agathis (1),...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# Rule-based blocking over input tables
# first get features that can be used
feature_table = em.get_features_for_blocking(rl, ar)

# CME:  Hmm.. how exactly are these features populated?  What is a 
# left attribute vs a right attribute?

In [15]:
len(feature_table)

20

In [16]:
#list name related features
feature_table[feature_table.left_attribute=='name']

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source,is_auto_generated
4,name_name_jac_qgm_3_qgm_3,name,name,qgm_3,qgm_3,jaccard,<function name_name_jac_qgm_3_qgm_3 at 0x10c0b2e18>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
5,name_name_cos_dlm_dc0_dlm_dc0,name,name,dlm_dc0,dlm_dc0,cosine,<function name_name_cos_dlm_dc0_dlm_dc0 at 0x10c0b2d08>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
6,name_name_jac_dlm_dc0_dlm_dc0,name,name,dlm_dc0,dlm_dc0,jaccard,<function name_name_jac_dlm_dc0_dlm_dc0 at 0x10c0fa378>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
7,name_name_mel,name,name,None,None,monge_elkan,<function name_name_mel at 0x10c0fa620>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
8,name_name_lev_dist,name,name,None,None,lev_dist,<function name_name_lev_dist at 0x10c0fa510>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
9,name_name_lev_sim,name,name,None,None,lev_sim,<function name_name_lev_sim at 0x10c0fa598>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
10,name_name_nmw,name,name,None,None,needleman_wunsch,<function name_name_nmw at 0x10c0fa488>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
11,name_name_sw,name,name,None,None,smith_waterman,<function name_name_sw at 0x10c0fa6a8>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True


In [19]:
# CME -- I'm realizing now that we may want to start with the attribute equivalence
# blocker for names and see where that leaves us.  On my TO-DO.
equivB = em.AttrEquivalenceBlocker()
C0 = equivB.block_tables(rl, ar, 'name', 'name',
                    l_output_attrs=['name', 'genus'], 
                    r_output_attrs=['name', 'genus'])
C0

# lolwat this reveals zero matches?


,_id,ltable_id,rtable_id,ltable_genus,rtable_genus
0,0,4114,368,Pristimantis,Pristimantis
1,1,4123,368,Pristimantis,Pristimantis
2,2,4126,368,Pristimantis,Pristimantis
3,3,4127,368,Pristimantis,Pristimantis
4,4,4128,368,Pristimantis,Pristimantis
5,5,4129,368,Pristimantis,Pristimantis
6,6,4141,368,Pristimantis,Pristimantis
7,7,4149,368,Pristimantis,Pristimantis
8,8,4156,368,Pristimantis,Pristimantis
9,9,4171,368,Pristimantis,Pristimantis


In [32]:
#Do overlap blocking on name to get candidate set
#Perform some kind of blocking to find candidate pairs
ob = em.OverlapBlocker()
C0 = ob.block_tables(rl, ar,'name', 'name', 
                     l_output_attrs=['name', 'genus'], 
                     r_output_attrs=['name', 'genus'])
C0

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:00


,_id,ltable_id,rtable_id,ltable_name,ltable_genus,rtable_name,rtable_genus
0,0,7316,0,Coahuilix De Hubbs Snail,Coahuilix,Piquero de Abbott,Sulidae
1,1,4580,0,"Carbonera Pupfish, Perrito de Carbonera",Cyprinodon,Piquero de Abbott,Sulidae
2,2,7546,0,Lago de las Minas Frog,Lithobates,Piquero de Abbott,Sulidae
3,3,4409,0,Moravia de Chirripo Salamander,Bolitoglossa,Piquero de Abbott,Sulidae
4,4,5809,3,Torreya Pygmy Grasshopper,Tettigidea,pygmy bluetongue,Scincidae
5,5,207,3,Oxleyan Pygmy Perch,Nannoperca,pygmy bluetongue,Scincidae
6,6,5154,3,Dwarf Pygmy Goby,Pandaka,pygmy bluetongue,Scincidae
7,7,5274,3,"Nguru Pygmy Chameleon, Nguru Spiny Pygmy Chameleon",Rhampholeon,pygmy bluetongue,Scincidae
8,8,7894,3,Pygmy Hog Sucking Louse,Haematopinus,pygmy bluetongue,Scincidae
9,9,2906,3,"Green Pygmy Chameleon, Pare Pygmy Chameleon",Rhampholeon,pygmy bluetongue,Scincidae


In [33]:
#Only keep candidate pairs which share genus
ab = em.AttrEquivalenceBlocker()
C1 = ob.block_candset(C0, 'genus', 'genus')
C1

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:00


,_id,ltable_id,rtable_id,ltable_name,ltable_genus,rtable_name,rtable_genus
19,19,2777,10,"Bornean Agile Gibbon, Bornean White-bearded Gibbon",Hylobates,Agile gibbon,Hylobates
377,377,1515,22,"Andes Smooth Frog, Lake Junin Frog, Marbled Water Frog",Telmatobius,Andes smooth frog,Telmatobius
424,424,4977,22,Hunaco Water Frog,Telmatobius,Andes smooth frog,Telmatobius
463,463,7524,22,Titicaca Water Frog,Telmatobius,Andes smooth frog,Telmatobius
542,542,4108,22,Amable Maria Frog,Telmatobius,Andes smooth frog,Telmatobius
620,620,5807,22,Arico Water Frog,Telmatobius,Andes smooth frog,Telmatobius
630,630,1454,22,Zapahuira Water Frog,Telmatobius,Andes smooth frog,Telmatobius
889,889,55,27,"Light-necked Sportive Lemur, Microdon Sportive Lemur, Small-toothed Sportive Lemur",Lepilemur,Ankarana sportive lemur,Lepilemur
903,903,189,27,Ankarana Sportive Lemur,Lepilemur,Ankarana sportive lemur,Lepilemur
907,907,53,27,Milne-Edwards's Sportive Lemur,Lepilemur,Ankarana sportive lemur,Lepilemur


In [ ]:
#For shits and giggles: 